In [9]:
import tensorflow as tf
print(tf.__version__)

2.9.2


In [10]:
device_name = tf.test.gpu_device_name()
print(device_name)

/device:GPU:0


In [11]:
with tf.device(device_name):
  if(device_name):
    print("GPU Enable !")
  else:
    print("GPU Not Enable !")

GPU Enable !


In [12]:
list_of_physical_devices = tf.config.experimental.list_physical_devices()
for i in list_of_physical_devices:
  print(i)

PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')
PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')


In [13]:
with_cuda = tf.test.is_built_with_cuda()
if(with_cuda):
  print("CUDA Present !")
else:
  print("CUDA Not present !")

CUDA Present !


In [14]:
strategy = tf.distribute.MirroredStrategy()

In [15]:
no_of_workers = strategy.num_replicas_in_sync
print("Number of Workers/GPU cores: ",no_of_workers)

Number of Workers/GPU cores:  1


### Model Creation and Dataset Collection

In [16]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

x_train_scaled = x_train/255.0
x_test_scaled = x_test/255.0

import keras
y_train_categorical = keras.utils.to_categorical(y_train, num_classes=10, dtype='float32')
y_test_categorical = keras.utils.to_categorical(y_test, num_classes=10,dtype='float32')

def model_definition():
  model = keras.Sequential([
      keras.layers.Flatten(input_shape=(32,32,3)),
      keras.layers.Dense(3000,activation='relu'),
      keras.layers.Dense(1000,activation='relu'),
      keras.layers.Dense(10,activation='sigmoid')
  ])
  model.compile(optimizer='sgd',loss='categorical_crossentropy',
                metrics='acc')
  return model
model = model_definition()
# print(model.summary())

train_tf_dataset = tf.data.Dataset.from_tensor_slices((x_train_scaled,y_train_categorical))
test_tf_dataset = tf.data.Dataset.from_tensor_slices((x_test_scaled,y_test_categorical))

170498071/170498071 [==============================] - 6s 0us/step


In [17]:
batch_size_per_replica = int(input("Enter Batch size to each replica: "))
batch_size = batch_size_per_replica * no_of_workers

Enter Batch size to each replica: 1000


In [18]:
train_dataset = train_tf_dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)
test_dataset = train_tf_dataset.batch(batch_size)

In [19]:
with strategy.scope():
  model = model_definition()
  model.fit(train_dataset, epochs=5)

Epoch 1/5
50/50 [==============================] - 5s 22ms/step - loss: 2.1315 - acc: 0.2356
Epoch 2/5
50/50 [==============================] - 1s 24ms/step - loss: 1.9695 - acc: 0.3215
Epoch 3/5
50/50 [==============================] - 1s 21ms/step - loss: 1.9004 - acc: 0.3466
Epoch 4/5
50/50 [==============================] - 1s 19ms/step - loss: 1.8581 - acc: 0.3603
Epoch 5/5
50/50 [==============================] - 1s 19ms/step - loss: 1.8279 - acc: 0.3705


In [20]:
with strategy.scope():
  prediction = model.predict(test_dataset)
print(prediction.shape)
import numpy as np
prediction = np.array([np.argmax(i) for i in prediction])
print(prediction.shape)

50/50 [==============================] - 3s 14ms/step
(50000, 10)
(50000,)


                                            -: END :-